In [1]:
from pprint import pprint

In [2]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Me fale mais sobre {topico}")

prompt_template.invoke({"topico": "Carros"})

StringPromptValue(text='Me fale mais sobre Carros')

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "Você é um concierge"),
    ("user", " Preciso de ajuda com: {topic}")
])

pprint(prompt_template.invoke({"topic": "Previsão do tempo"}).to_messages())

[SystemMessage(content='Você é um concierge', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' Preciso de ajuda com: Previsão do tempo', additional_kwargs={}, response_metadata={})]


In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "Você é um concierge"),
    MessagesPlaceholder("mensagens")
])

mensagem = prompt_template.invoke({"mensagens": [HumanMessage(content="Preciso de ajuda com a previsão do tempo"),HumanMessage(content="e também sobre o caminho mais rápido pro trabalho")]})
pprint(mensagem.to_messages())

[SystemMessage(content='Você é um concierge', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Preciso de ajuda com a previsão do tempo', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='e também sobre o caminho mais rápido pro trabalho', additional_kwargs={}, response_metadata={})]


In [5]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

template_completo = """{introducao}

{exemplo}

{comeco}"""
prompt_completo = PromptTemplate.from_template(template_completo)

template_introducao = """Você é um {profissao}."""
prompt_introducao = PromptTemplate.from_template(template_introducao)

template_exemplo = """Aqui está um exemplo de uma interação:

P: {pergunta_exemplo}
R: {resposta_exemplo}"""
prompt_exemplo = PromptTemplate.from_template(template_exemplo)

template_comeco = """Agora responda a próxima pergunta

P: {pergunta}
R:"""
prompt_comeco = PromptTemplate.from_template(template_comeco)

prompts_entrada = [
    ("introducao", prompt_introducao),
    ("exemplo", prompt_exemplo),
    ("comeco", prompt_comeco),
]
pipeline = PipelinePromptTemplate(
    final_prompt=prompt_completo, pipeline_prompts=prompts_entrada
)

pipeline.input_variables

/tmp/ipykernel_38381/763714013.py:30: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline = PipelinePromptTemplate(


['pergunta_exemplo', 'resposta_exemplo', 'profissao', 'pergunta']

In [6]:
print(
    pipeline.format(
        profissao="Cozinheiro",
        pergunta_exemplo="Qual a sua torta favorita?",
        resposta_exemplo="A torta de banana!",
        pergunta="E qual é o seu bolo favorito?",
    )
)

Você é um Cozinheiro.

Aqui está um exemplo de uma interação:

P: Qual a sua torta favorita?
R: A torta de banana!

Agora responda a próxima pergunta

P: E qual é o seu bolo favorito?
R:


In [7]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

exemplos = [
    {"input": "Qual o meu saldo?", "output": "Pergunta"},
    {"input": "Quero fazer uma transferência", "output": "Ação"},
]

prompt_exemplos = ChatPromptTemplate.from_messages(
    [
        ("human", "Mensagem: {input}"),
        ("ai", "Categoria: {output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt_exemplos,
    examples=exemplos,
)

pprint(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='Mensagem: Qual o meu saldo?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Categoria: Pergunta', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Mensagem: Quero fazer uma transferência', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Categoria: Ação', additional_kwargs={}, response_metadata={})]


In [8]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um decisor que precisa classificar mensagens em duas categorias, pergunta ou ação"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
pprint(final_prompt.invoke("Preciso ver meu saldo").to_messages())

[SystemMessage(content='Você é um decisor que precisa classificar mensagens em duas categorias, pergunta ou ação', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Mensagem: Qual o meu saldo?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Categoria: Pergunta', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Mensagem: Quero fazer uma transferência', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Categoria: Ação', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Preciso ver meu saldo', additional_kwargs={}, response_metadata={})]
